# Check For 64-bit

In [1]:
import platform
platform.architecture()[0]

'64bit'

# Load Libraries

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from igraph import *
import math
from rtree import index
import math
import matplotlib.pyplot as plt
import csv
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_samples, silhouette_score
from datetime import timedelta
import datetime
import pytz
import fiona
from shapely.geometry import LineString, Point, shape, mapping
from dateutil import parser
from fiona.crs import from_epsg
from sklearn.metrics import accuracy_score
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from scipy import spatial
from scipy.sparse import coo_matrix
from scipy.sparse import csgraph
%matplotlib inline

# Develop graph from input points for calculating network distances

In [ ]:
class graphFromLinesAndPoints(object):
    
    def __init__(self,indexAndPoints):
        self.nodeIDPoints =indexAndPoints
        self.G = None#Graph()#nx.Graph()
        self.K  = 10
        self.epsilon = .001
        self.allNodesMatch = {}
        self.allNodesMatchWithLines = {}
        self.rtIndex = None
        self.weights = []
        self.nodesTemporary = []
        self.edgesTemporary = []
        self.pntIndexToNodeID = {}
        self.NodeIDtopntIndex = {}
        self.pntLst = []
        self.lowerDM = None
        self.baseforsparse = []
        self.pntMat = None
        
    def distance(self,x1,y1,x2,y2):
        dx = x1-x2
        dy = y1-y2
        return math.sqrt(dx*dx+dy*dy)
    
    def distanceNode(self,node1,node2):
        dx = node1[0]-node2[0]
        dy = node1[1]-node2[1]
        return math.sqrt(dx*dx+dy*dy)
    
    #need to contend with duplicate points...
    def developGraphFromTriangulation(self):
        self.pntLst = range(0,len(self.nodeIDPoints.keys()))
        self.pntIndexToNodeID = {}
        self.NodeIDtopntIndex = {}
        print "Remove Duplicates"
        for i,k in enumerate(self.nodeIDPoints.keys()):
            self.pntLst[i] = self.nodeIDPoints[k]
            
        
            
        XC = np.array(self.pntLst)
        self.pointMat = XC
        print "creating triangulation"
        tri = spatial.Delaunay(XC)
        data_l = []
        col_l = []
        row_l = []
        existingPairs = []
        ids = []
        
        for t in tri.simplices:
            
            ds = spatial.distance.pdist(np.vstack([XC[t[0]],XC[t[1]],XC[t[2]]]),'euclidean')
            ids.extend(t)
            self.weights.append(ds[0])
            self.weights.append(ds[1])
            self.weights.append(ds[2])
            pair = (t[0],t[1])
            if not pair in existingPairs:
                row_l.append(t[0])
                col_l.append(t[1])
                data_l.append(ds[0])
                existingPairs.append(pair)
            pair = (t[0],t[2])
            if not pair in existingPairs:
                row_l.append(t[0])
                col_l.append(t[2])
                data_l.append(ds[1])
                existingPairs.append(pair)
            pair = (t[1],t[2])
            if not pair in existingPairs:
                row_l.append(t[1])
                col_l.append(t[2])
                data_l.append(ds[2])
                existingPairs.append(pair)
                
        self.baseforsparse = zip(row_l,col_l,data_l)
        

        print "creating spatial index from unique points"
        triids = list(set(ids))
        self.rtIndex = spatial.KDTree(XC[triids])
        print "match points to their index"
        
        for i,k in enumerate(self.nodeIDPoints.keys()):
            pnt = np.array(self.nodeIDPoints[k])
            trq = self.rtIndex.query(pnt,k=1)
            closestID = triids[trq[1]]
            self.pntIndexToNodeID[closestID]=k
            self.NodeIDtopntIndex[k]=closestID

        print "Sparse Matrix"
        self.G = coo_matrix((np.array(data_l),(np.array(row_l),np.array(col_l))),shape=(len(XC),len(XC)))
        
        print "Lower Distance Matrix"
        self.lowerDM = np.tril(csgraph.dijkstra(self.G,directed=False))                     

        
    def getNodeDistances(self,nodeID1,nodeID2):
        ind1 = self.NodeIDtopntIndex[nodeID1]
        ind2 = self.NodeIDtopntIndex[nodeID2]
        if ind1 > ind2:
            dist = self.lowerDM[ind1][ind2]
        else:
            dist = self.lowerDM[ind2][ind1]
        if dist == 0 or dist<0 or dist==np.Inf:
            return None
        else:
            return dist
    
        return None
    
    
    def getMaxKNearest(self,nodeID,K):

        i = self.NodeIDtopntIndex[nodeID]

        stuff = [x for x in self.lowerDM[i+1:,i] if x != np.inf]+[x for x in self.lowerDM[i,0:i] if x != np.inf]

        stuff = sorted(stuff)
 
        if max(stuff[0:K]) == 0:
            return 1.0

        return max(stuff[0:K])
    
    def getMaxKNearDict(self,keys, K):
        maxLst = {}
        for n in keys:
            maxLst[n] = self.getMaxKNearest(n,K)
        return maxLst

# Hierarchical clustering using linear weighted combination of Space Time and Attribute values

## Based on distance matrix approach in Scipy

In [ ]:
class st_hierarchical_net(object):
    
    def __init__(self):
        self.K = 10
        self.pointFeatureClass = ""#path to pointFeature Class
        self.gdbPath = ""
        self.outputPathFC = ""
        self.outputNameFC = ""
        self.timeScale = 86400 #used in the coversion to a nonlinear function
        self.timeByScale = 3600 #Scale data to this value
        self.timeCenter = 43200
        self.attribScale = 1
        self.idxTreeNodeID = {}
        self.timeField = ""
        self.attribField = ""
        self.indexPointID = {}
        self.maxKNearest = {}
        self.df = None #pandas data frame
        self.dm = None #sparse distance matrix
        self.dfdm = None #pandas data frame of distance matrix
        self.row_clusters = None
        self.local_time_zone = None
        self.weights = []
        self.useridfield = None
        self.indexidfield = None
        self.indexuseridmatch = {}
        self.labels = []
        
    def loadDataset(self):
        

        variables = ['NodeID', 'Time','Attribute']
        self.labels = []
        x_lst = []
        idx = 0
        nodeIds = {}

        with fiona.open(self.gdbPath,'r',layer=self.pointFeatureClass) as coll:
            i = 1
            for row in coll:
                s = shape(row['geometry'])
                indexValue = None
                if self.indexidfield:
                    self.labels.append(row['properties'][self.indexidfield])
                    indexValue = row['properties'][self.indexidfield]
                else:
                    self.labels.append(i)
                    indexValue = i
                    i+=1
                    
                nodeIds[indexValue]={}
                dt = row['properties'][self.timeField]
                x_lst.append([int(indexValue),dt,row['properties'][self.attribField]])
                
                self.indexPointID[indexValue] = (s.coords[0][0],s.coords[0][1])
                if self.useridfield:
                    self.indexuseridmatch[indexValue] = row['properties'][self.useridfield]
                #idx+=1
        self.internalGraph = graphFromLinesAndPoints(self.indexPointID)
        self.internalGraph.developGraphFromTriangulation()
        self.totalNetworkLength = sum(self.internalGraph.weights)
        
        X = np.array(x_lst)
        self.df = pd.DataFrame(X, columns=variables, index=self.labels)
        self.df['NodeID'] = self.df['NodeID'].astype(int)
        print "get max k"
        self.maxKNearest = self.internalGraph.getMaxKNearDict(self.internalGraph.NodeIDtopntIndex.keys(),self.K)
        
        print "Creating distance matrix"
        print X
        self.dm = pdist(X, self.calcDistanceMetric)
        print "create clusters"
        self.row_clusters = linkage(self.dm,method='average')
        plt.title('Hierarchical Clustering Dendrogram')
        plt.xlabel('Index')
        plt.ylabel('Distance')    
        
    def calcDistanceMetric(self,x,y):
        #assumed that there are three dimensions        
        nearestMax = self.maxKNearest[int(x[0])]

        dist = self.internalGraph.getNodeDistances(int(x[0]),int(y[0]))
        if dist == None:
            dist = self.totalNetworkLength /2.0

        distSc = float(dist) / nearestMax#(float(self.totalNetworkLength)/2.0)
        
        t1 = x[1] - self.timeCenter #centered to noon, assuming seconds from midnight
        t2 = y[1] - self.timeCenter #centered to noon, assuming seconds from midnight
        
        a1 = x[2]
        a2 = y[2]
        
        
        if self.weights[0]*x[0] == self.weights[0]*y[0] and self.weights[1]*t1==self.weights[1]*t2 and self.weights[2]*a1==self.weights[2]*a2:
            #print "equals"
            return 0.0

        
        tSec = abs(t1 - t2) 
        tRad = math.pi * (float(tSec)/self.timeScale) #convert time to radians, half unit circle
        tCos = math.cos(tRad)
        tDist = math.sqrt((t1*t1+t2*t2)-(2*t1*t2*tCos))#law of cosines to convert it to euclidean space
        da = np.abs(x[2]-y[2])
        daSc = da / self.attribScale
        dtSc = float(tDist)/self.timeByScale #scaled to a value if in seconds 3600 is seconds from midnight
        res = float(self.weights[0]*distSc + self.weights[1]*dtSc+ self.weights[2]*daSc)/(sum(self.weights))

        if res >=0.0:
            return res
        else:
            return 0.0
    
    def datetime_to_float(self,d):
        epoch_utc = pytz.utc
        epoch = datetime.datetime.utcfromtimestamp(0)
        epochdt = epoch_utc.localize(epoch,is_dst=None)
        total_seconds =  (d - epochdt).total_seconds()
        
        # total_seconds will be in decimals (millisecond precision)
        return total_seconds

    def float_to_datetime(self,fl):
        return datetime.datetime.utcfromtimestamp(fl)
    
    def silohetteScore(self,n_clusters,printV=False):
        outScores = []
        sqdm = squareform(self.dm)
        for n_c in n_clusters:
            try:
                cluster_labels = fcluster(self.row_clusters, n_c, criterion='maxclust')
                silhouette_avg = silhouette_score(sqdm,cluster_labels,metric='precomputed')
                outScores.append(silhouette_avg)
                if printV:
                    print "For n_clusters =%s"%n_c
                    print "The average silhouette_score is : %s"%silhouette_avg
            except:
                pass
        del sqdm
        return outScores

In [ ]:
%matplotlib inline
sth = st_hierarchical_net()
sth.K = 7
sth.pointFeatureClass = "jimmy_2_6"#"comparisonDataset_20"#"allgeo_validsources_utm_random_la_sm"# "allgeo_validsources_utm_noident"#path to pointFeature Class
sth.gdbPath = r"C:\Users\David\OneDrive\projects\STHC\dataset.gdb"#r"C:\Users\David\OneDrive\projects\SNN-Net\simWorking\sim_db.gdb"#r"C:\Users\David\OneDrive\projects\dissertationWork\data\allGeoTaggedMessages.gdb"#r"C:\Users\David\OneDrive\projects\dissertationWork\data\allGeoTaggedMessages.gdb"
sth.outputPathFC = ""
sth.outputNameFC = ""
sth.timeScale = 86400
sth.timeByScale = 3600#t #standard deviation may provide a good way to set this
sth.attribScale = 1
sth.timeField = "TOTSEC"#"time"#"secfrmmid" #assumed to be string since shapefiles cannot handle datetimes
sth.attribField ="ATTR"#"ATTRIB"#"attrib"
sth.weights=[.2,1.0,0.0]
#sth.useridfield = "ATTR"
#sth.indexidfield = "Try"
print "process data"
sth.loadDataset()
#tracker[t][k]['sth'] = sth+
#tracker[t][k]['scores']= sth.silohetteScore(cs)